In [0]:
%python
dbutils.data.help()

### 1. Load the data from volume to dataframe

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("loadAgeListtoWidget").getOrCreate()
csv_df = spark.read.csv('/Volumes/workspace/wd36schema/ingestion_volume/target/csvout',header=True).where("Age is not null")
display(csv_df)

### 2. DSL-Transfomration to derive the column (ageList) 

In [0]:
from pyspark.sql import functions as F
df = csv_df.withColumn("ageList", 
                       F.when( (F.col("age")>=-1) & (F.col("age")<=35),'Kids')
                       .when( (F.col("age")>35) & (F.col("age")<=40),'Adult')
                       .when( (F.col("age")>40) & (F.col("age")<=60),'Aged')
                       .otherwise(None)).where("ageList is not null")


### 3. Convert the dataframe to string values

In [0]:
df =df.select("ageList").distinct()
lst = df.collect()
ageList = [str(item.ageList) for item in lst]
print(ageList)


### 4. Store the column value to widgets object

In [0]:
dbutils.widgets.combobox(
  name='ageWiseLst_dsl',
  defaultValue='Kids',
  choices=ageList,
  label='AgeList'
)
print(dbutils.widgets.get("ageWiseLst_dsl"))

### 5. SQL-Transfomration to derive the column (ageList)

In [0]:

csv_df.createOrReplaceTempView("vu")

new_df = spark.sql("""select distinct case when age between 30 and 35 then 'Kids' when age between 36 and 40 then 'Adult' when age between 41 and 60 then 'Aged' else null end as agelist from vu""").where("agelist is not null")

lst = list(new_df.collect())

choices_list = [str(row.agelist) for row in lst]

print(choices_list)

### 6. Store the column value to widgets object

In [0]:
dbutils.widgets.combobox(
  name='ageWiseLst_sql',
  defaultValue='Kids',
  choices=choices_list,
  label='AgeList'
)
print(dbutils.widgets.get("ageWiseLst_sql"))

In [0]:
import datetime
dt =datetime.datetime.now().strftime("%Y%m%d%H%M%S")
print(dt)

dbutils.fs.put(f"/Volumes/workspace/wd36schema/ingestion_volume/target/{dt}_hello.txt", "Hello, Databricks!", False)


In [0]:

filesLst = dbutils.fs.ls("/Volumes/workspace/wd36schema/ingestion_volume/target/")
for file in filesLst:
    print(file.name)


In [0]:
filesLst = dbutils.fs.rm("/Volumes/workspace/wd36schema/ingestion_volume/target/20251227022714_hello.txt");

print(filesLst)